# Curate_V1_PassiveFeatures_data

In [4]:
%matplotlib inline

import datetime as dt
import itertools as it

from IPython.core.interactiveshell import InteractiveShell
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import synapseclient
from synapseclient import Activity, Schema, Table, as_table_columns

InteractiveShell.ast_node_interactivity = 'all'
syn = synapseclient.Synapse()
syn.login()

def isnum(x):
    if x is None:
        return False
    try:
        float(x)
        return True
    except ValueError:
        return False

def safe_round(x):
    if not pd.isnull(x) and isnum(x):
        return np.round(x, 3)
    else:
        return x

Welcome, Abhishek Pratap!



### Prepare V1 data

In [5]:
v1sid = 'syn10236538'
v1r = pd.read_csv(syn.get(v1sid).path, sep='\t')
v1r.head()

,user_id,study_arm,unreturned_calls,mobility,sms_length,call_duration,interaction_diversity,missed_interactions,aggregate_communication,sms_count,mobility_radius,call_count,passive_date_pacific,User_Phone_Type,Cohort,brightenid,day,week,start
0,uid10431,BLUE,NaN,0.112392,NaN,NaN,NaN,NaN,NaN,NaN,3087.887243,NaN,2014-08-05,iPhone,PST,BLUE-00048,5.0,1.0,2014-08-01
1,uid10431,BLUE,NaN,0.413090,NaN,NaN,NaN,NaN,NaN,NaN,21.643242,NaN,2014-08-03,iPhone,PST,BLUE-00048,3.0,1.0,2014-08-01
2,uid10431,BLUE,NaN,1.099014,NaN,NaN,NaN,NaN,NaN,NaN,5.894548,NaN,2014-08-12,iPhone,PST,BLUE-00048,12.0,2.0,2014-08-01
3,uid10431,BLUE,NaN,4.690892,NaN,NaN,NaN,NaN,NaN,NaN,1.453950,NaN,2014-08-02,iPhone,PST,BLUE-00048,2.0,1.0,2014-08-01
4,uid10431,BLUE,NaN,0.002924,NaN,NaN,NaN,NaN,NaN,NaN,0.001462,NaN,2014-08-06,iPhone,PST,BLUE-00048,6.0,1.0,2014-08-01


In [ ]:
v1 = v1r.drop(columns=[
    'user_id', 'study_arm', 'start'
]).rename(columns={
    'brightenid':'participant_id',
    'passive_date_pacific':'dt_passive'
})

v1.columns = [c.lower() for c in v1.columns]

v1.head()

In [ ]:
toround = list(set(v1.columns) - {'dt_response', 'user_phone_type', 'cohort', 'participant_id', 'day', 'week'})
for c in toround:
    v1[c] =v1[c].apply(safe_round)

v1.head()

In [ ]:
colorder = ['participant_id', 'dt_passive', 'day', 'week', 'cohort', 'user_phone_type']
colorder = colorder + sorted(list(set(v1.columns) - set(colorder)))
v1 = v1[colorder]

v1.head()

In [ ]:
v1.mobility.hist()
v1.mobility.describe()

In [ ]:
'an element $$\(\in\)$$ {' + ', '.join([f'_{c}_' for c in v1.user_phone_type.unique() if str(c) != 'nan']) + '}'

In [ ]:
final = syn.store(Table(
    Schema(
            name='V1 Passive Features',
            columns=as_table_columns(v1), 
            parent='syn10848316'),
        v1
    )
)

In [6]:
final = syn.setProvenance(
    'syn17025500',
    activity=Activity(
        name='Generate V1 Table Data',
        description='Process the data collected during study ',
        used=[v1sid],
        executed=[
            dict(
                name='Curate_V1_PassiveFeatures_data',
                url='https://github.com/apratap/BRIGHTEN-Data-Release/blob/master/Curate_V1_PassiveFeatures_data.ipynb'
            )
        ]
    )
)